In [1]:
from torchtext.datasets import IMDB

train_iter = IMDB(split='train')

/Users/mkhokhlush/github/pytorch-ml-experiments/.venv/lib/python3.10/site-packages/torch/utils/data/datapipes/utils/common.py:24: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn(
/Users/mkhokhlush/github/pytorch-ml-experiments/.venv/lib/python3.10/site-packages/torch/utils/data/datapipes/iter/selecting.py:54: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn("Lambda function is not supported for pickle, please use "


In [35]:
for label, line in train_iter:
    print(label, line)
    break

neg I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between,

### Tokenizer and Vocab

In [19]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

/Users/mkhokhlush/github/pytorch-ml-experiments/.venv/lib/python3.10/site-packages/torch/utils/data/datapipes/iter/combining.py:180: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [28]:
print(tokenizer("here is an example the fdfaf"))
print(vocab(['here', 'is', 'an', 'example', 'the', 'fdfaf']))
vocab.get_itos()[:10]

['here', 'is', 'an', 'example', 'the', 'fdfaf']
[131, 9, 40, 464, 1, 0]


['<unk>', 'the', '.', ',', 'and', 'a', 'of', 'to', "'", 'is']

In [36]:
text_pipeline = lambda line: vocab(tokenizer(line))
label_pipeline = lambda label: 0 if label == "neg" else 1

for label, line in train_iter:
    print(text_pipeline(line))
    print(label_pipeline(label))
    break

[12, 1567, 12, 245, 35467, 42, 63, 397, 1134, 91, 6, 36, 1, 7125, 14, 3362, 10, 59, 10, 16, 93, 628, 11, 6920, 2, 12, 86, 552, 14, 37, 93, 10, 16, 20192, 39, 1224, 2, 15, 2, 9262, 50, 10, 130, 779, 7, 2479, 13, 681, 3, 1574, 117, 5, 341, 6, 113, 1159, 3051, 12, 71, 74, 7, 73, 13, 18, 536, 2, 1, 120, 9, 5958, 193, 5, 190, 3861, 473, 1423, 765, 4313, 41, 488, 7, 833, 286, 60, 57, 49, 126, 2, 11, 825, 60, 488, 7, 1131, 46, 11858, 7, 256, 55, 440, 6, 668, 27, 53, 1, 862, 29736, 208, 49, 780, 1000, 1303, 146, 17, 1, 2674, 336, 4, 1509, 1303, 11, 1, 2358, 1591, 2, 11, 202, 2181, 7270, 4, 1918, 19585, 6, 21477, 49, 72, 4655, 27, 2380, 3, 60, 51, 401, 19, 46, 473, 1691, 3, 8134, 3, 4, 998, 346, 2, 53, 1079, 77, 49, 12, 245, 35467, 9, 14, 1613, 160, 586, 3, 13, 16, 1159, 8205, 2, 71, 3, 1, 401, 4, 999, 144, 30, 174, 4, 241, 202, 3, 62, 100, 10, 8, 15, 28, 329, 44, 55, 6654, 99, 4460, 2, 142, 63, 23464, 347, 171, 10, 1573, 3, 11, 634, 401, 4, 999, 30, 5, 662, 10197, 11, 3861, 436, 2, 62, 14515, 

### Dataloader

In [38]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

### Model

In [43]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, 1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize).to(device)
vocab_size

100683

### Training

In [44]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        probability = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        predicted_label = 1 if probability >= 0.5 else 0
        total_acc += (predicted_label == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            probability = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)